# 1. Get source data

In [1]:
# %load ../src/data/get_data.py

import subprocess
import os

import pandas as pd
import numpy as np

from datetime import datetime

import requests
import json


def get_john_hopkins_data():
    ''' 
        Git repo John Hopkins data : https://github.com/CSSEGISandData/COVID-19.git
        Get john hopkins data by a git pull request
        Result is stored in the predefined csv structure
    '''
    git_pull = subprocess.Popen("/usr/bin/git pull",
                                cwd=os.path.dirname('../data/raw/COVID-19/'),
                                shell=True,
                                stdout=subprocess.PIPE,
                                stderr=subprocess.PIPE)
    (out, error) = git_pull.communicate()

    print("Error : " + str(error))
    print("out : " + str(out))


if __name__ == '__main__':
    get_john_hopkins_data()
    


Error : b'From https://github.com/CSSEGISandData/COVID-19\n   29b85c49..5d49f1e6  master     -> origin/master\n   23632359..b54d1130  web-data   -> origin/web-data\n'
out : b'Updating 29b85c49..5d49f1e6\nFast-forward\n README.md | 1 +\n 1 file changed, 1 insertion(+)\n'


# 2. Process Pipeline

In [12]:
# %load ../src/data/process_JH_data.py
import pandas as pd
import numpy as np

from datetime import datetime


def transform_relational_JH_data():
    ''' Transforms the COVID data into a relational data set

    '''

    data_path = '../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
    pd_raw = pd.read_csv(data_path)

    pd_data_base = pd_raw.rename(columns={'Country/Region': 'country',
                                          'Province/State': 'state'})

    pd_data_base['state'] = pd_data_base['state'].fillna('no')

    pd_data_base = pd_data_base.drop(['Lat', 'Long'], axis=1)

    pd_relational_model = pd_data_base.set_index(['state', 'country']) \
        .T                              \
        .stack(level=[0, 1])             \
        .reset_index()                  \
        .rename(columns={'level_0': 'date',
                         0: 'confirmed'},
                )

    pd_relational_model['date'] = pd_relational_model.date.astype(
        'datetime64[ns]')

    pd_relational_model.to_csv(
        '../data/processed/COVID_relational_confirmed.csv', sep=';', index=False)
    print(' Number of rows stored: ' + str(pd_relational_model.shape[0]))


if __name__ == '__main__':

    transform_relational_JH_data()


 Number of rows stored: 59850


# 3. Filter data and doubling rate calculation

In [5]:
# %load ../src/features/build_features.py
import numpy as np
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)
import pandas as pd

from scipy import signal


def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate

        Parameters:
        ----------
        in_array : pandas.series

        Returns:
        ----------
        Doubling rate: double
    '''

    y = np.array(in_array)
    X = np.arange(-1, 2).reshape(-1, 1)

    assert len(in_array) == 3
    reg.fit(X, y)
    intercept = reg.intercept_
    slope = reg.coef_

    return intercept / slope


def savgol_filter(df_input, column='confirmed', window=5):
    ''' Savgol Filter which can be used in groupby apply function (data structure kept)

        parameters:
        ----------
        df_input : pandas.series
        column : str
        window : int
            used data points to calculate the filter result

        Returns:
        ----------
        df_result: pd.DataFrame
            the index of the df_input has to be preserved in result
    '''

    degree = 1
    df_result = df_input

    filter_in = df_input[column].fillna(0)

    result = signal.savgol_filter(np.array(filter_in),
                                  window,  # window size used for filtering
                                  1)
    df_result[str(column + '_filtered')] = result
    return df_result


def rolling_reg(df_input, col='confirmed'):
    ''' Rolling Regression to approximate the doubling time'

        Parameters:
        ----------
        df_input: pd.DataFrame
        col: str
            defines the used column
        Returns:
        ----------
        result: pd.DataFrame
    '''
    days_back = 3
    result = df_input[col].rolling(
        window=days_back,
        min_periods=days_back).apply(get_doubling_time_via_regression, raw=False)

    return result


def get_filtered_data(df_input, filter_on='confirmed'):
    '''  Calculate savgol filter and return merged data frame

        Parameters:
        ----------
        df_input: pd.DataFrame
        filter_on: str
            defines the used column
        Returns:
        ----------
        df_output: pd.DataFrame
            the result will be joined as a new column on the input data frame
    '''

    must_contain = set(['state', 'country', filter_on])
    assert must_contain.issubset(set(
        df_input.columns)), ' Error in get_filtered_data when not all columns in data frame'

    #get a copy here otherwise the filter_on column will be overwritten
    df_output = df_input.copy()

    pd_filtered_result = df_output[['state', 'country', filter_on]].groupby(
        ['state', 'country']).apply(savgol_filter)  

    df_output = pd.merge(df_output, pd_filtered_result[[str(
        filter_on + '_filtered')]], left_index=True, right_index=True, how='left')
    return df_output.copy()


def get_doubling_rate(df_input, filter_on='confirmed'):
    ''' Calculate approximated doubling rate and return merged data frame

        Parameters:
        ----------
        df_input: pd.DataFrame
        filter_on: str
            defines the used column
        Returns:
        ----------
        df_output: pd.DataFrame
            the result will be joined as a new column on the input data frame
    '''

    must_contain = set(['state', 'country', filter_on])
    assert must_contain.issubset(set(
        df_input.columns)), ' Error in get_filtered_data when not all columns in data frame'

    pd_DR_result = df_input.groupby(['state', 'country']).apply(
        rolling_reg, filter_on).reset_index()

    pd_DR_result = pd_DR_result.rename(columns={filter_on: filter_on + '_DR',
                                                'level_2': 'index'})

    # merge dataframes on the index of the big table and on the index column after groupby
    df_output = pd.merge(df_input, pd_DR_result[['index', str(
        filter_on + '_DR')]], left_index=True, right_on=['index'], how='left')
    df_output = df_output.drop(columns=['index'])

    return df_output


if __name__ == '__main__':
    
   # test_data_reg = np.array([2, 4, 6])
   # result = get_doubling_time_via_regression(test_data_reg)
   # print('the test slope is: ' + str(result))

    pd_JH_data = pd.read_csv(
        '../data/processed/COVID_relational_confirmed.csv', sep=';', parse_dates=[0])
    pd_JH_data = pd_JH_data.sort_values('date', ascending=True).copy()

    pd_result_larg = get_filtered_data(pd_JH_data)
    pd_result_larg = get_doubling_rate(pd_result_larg)
    pd_result_larg = get_doubling_rate(pd_result_larg, 'confirmed_filtered')

    mask = pd_result_larg['confirmed'] > 100
    pd_result_larg['confirmed_filtered_DR'] = pd_result_larg['confirmed_filtered_DR'].where(
        mask, other=np.NaN)
    pd_result_larg.to_csv(
        '../data/processed/COVID_final_set.csv', sep=';', index=False)
    print(pd_result_larg[pd_result_larg['country'] == 'Nepal'].tail())


            date state country  confirmed  confirmed_filtered  confirmed_DR  \
44995 2020-08-29    no   Nepal    37340.0             37469.2     40.244800   
44996 2020-08-30    no   Nepal    38561.0             38469.2     35.584165   
44997 2020-08-31    no   Nepal    39460.0             39507.8     36.277044   
44998 2020-09-01    no   Nepal    40529.0             40566.4     40.159214   
44999 2020-09-02    no   Nepal    41649.0             41625.0     37.045226   

       confirmed_filtered_DR  
44995              36.208275  
44996              37.309699  
44997              37.753426  
44998              37.683069  
44999              38.320801  


# 4. Prototype Visual Board

In [ ]:
# %load ../src/visualization/visualize.py
import pandas as pd
import numpy as np

import dash
dash.__version__
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State

import plotly.graph_objects as go

import os

df_input_large = pd.read_csv('../data/processed/COVID_final_set.csv', sep=';')


fig = go.Figure()

app = dash.Dash()
app.layout = html.Div([

    dcc.Markdown('''
    # COVID-19 Dashboard

    '''),

    dcc.Markdown('''
       #### Multi-Select Country for visualization
    '''),


    dcc.Dropdown(
        id='country_drop_down',
        options=[{'label': each, 'value': each}
                 for each in df_input_large['country'].unique()],
        value=['US', 'Germany', 'Nepal'],  # which are pre-selected
        multi=True
    ),

    dcc.Markdown('''
        #### Select Timeline of confirmed COVID-19 cases or the approximated doubling time
        '''),


    dcc.Dropdown(
        id='doubling_time',
        options=[
            {'label': 'Timeline Confirmed ', 'value': 'confirmed'},
            {'label': 'Timeline Confirmed Filtered', 'value': 'confirmed_filtered'},
            {'label': 'Timeline Doubling Rate', 'value': 'confirmed_DR'},
            {'label': 'Timeline Doubling Rate Filtered',
             'value': 'confirmed_filtered_DR'},
        ],
        value='confirmed',
        multi=False
    ),

    dcc.Graph(figure=fig, id='main_window_slope')
])


@app.callback(
    Output('main_window_slope', 'figure'),
    [Input('country_drop_down', 'value'),
     Input('doubling_time', 'value')])
def update_figure(country_list, show_doubling):

    if ('confirmed_DR' in show_doubling) or ('confirmed_filtered_DR' in show_doubling):
        my_yaxis = {'type': "log",
                    'title': 'Approximated doubling rate over 3 days (larger numbers are better #stayathome)'
                    }
    else:
        my_yaxis = {'type': "log",
                    'title': 'Confirmed infected people (source johns hopkins csse, log-scale)'
                    }

    traces = []
    for each in country_list:

        df_plot = df_input_large[df_input_large['country'] == each]

        if show_doubling == 'confirmed_filtered_DR':
            df_plot = df_plot[['state', 'country', 'confirmed', 'confirmed_filtered', 'confirmed_DR',
                               'confirmed_filtered_DR', 'date']].groupby(['country', 'date']).agg(np.mean).reset_index()
        else:
            df_plot = df_plot[['state', 'country', 'confirmed', 'confirmed_filtered', 'confirmed_DR',
                               'confirmed_filtered_DR', 'date']].groupby(['country', 'date']).agg(np.sum).reset_index()


        traces.append(dict(x=df_plot.date,
                           y=df_plot[show_doubling],
                           mode='markers+lines',
                           opacity=0.9,
                           name=each
                           )
                      )

    return {
        'data': traces,
        'layout': dict(
            width=1280,
            height=720,

            xaxis={'title': 'Timeline',
                   'tickangle': -45,
                   'nticks': 20,
                   'tickfont': dict(size=14, color="#7f7f7f"),
                   },

            yaxis=my_yaxis
        )
    }


if __name__ == '__main__':

    app.run_server(debug=True, use_reloader=False)


Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
